# Assignment 4

### Samuel Sheehy (Student ID 18143565)

### Techniques of High Performance Computing (PHAS0102)

### MSc Scientific Computing, University College London, 2019-2020

**This Jupyter Notebook is submitted as completion of the 4th Assignment for the UCL course cited above.**

## Setup

In [5]:
import numpy as np
from scipy.sparse import coo_matrix, linalg
import matplotlib.pyplot as plt
from math import ceil

TRUE_RESULT = 

## Introduction of Problem
* Intro to the problem we are solving and the Heat Equation 


## Mathematical Derivation of Numerical Scheme

* Finite difference method:

CN finite difference in spacial dimension.
$$
u_h(t, x, y) = u_h(t - \tau, x, y) -  \frac{\tau}{h^2} \left[ 4 u_h(t - \tau, x, y)
- u_h(t - \tau, x - h, y) - u_h(t - \tau, x + h, y) - u_h(t - \tau, x, y - h) - u_h(t - \tau, x, y + h) \right] + again
$$

* How I formulate this as a matrix


## My Method
* From mathematical formulation, components of calculation are: 
    - Construction of matrices A(t) and A(t+1)
    - Calculate $u_* = A(t)u(t) + [bc]$ - maybe use OpenCL kernel?
    - Solve for $u(t+1)$: $A(t+1)u(t+1) = u_* $ (CG method -  preconditioner through LU decomposition)
    - Use secant method to find answer

## Analysis
* Show influence of size of timestep and spacestep in finding correct answer
* How time completion of each section increases with system size
* Overall convergence of final solution

In [3]:
def is_bc(i, j, M):
    """
    Check if the given coordinates correspond to a boundary.
    """
    if i == 0 or i == M - 1:
        return True
    elif j == 0 or j == M - 1:
        return True
    else:
        return False

    
def show(b, M):
    p = b.reshape(M, M)
    p = np.pad(p, 1)

    # left bc
    p[:, 0] = 0

    # right bc
    p[:, -1] = 0

    # top bc
    p[-1, :] = 0

    # bottom bc
    p[0, :] = 5

    plt.imshow(p, origin='lower')
    plt.colorbar()
    plt.show()

# Boundaries:
def f(y, x):
    if y == 0: return 5
    else: return 0

In [66]:
# Crank-Nicholson Method
def build_CN_system(tau, h0):
    """
    Build a matrix which applies an explicit method to solve the problem.
    """
    data = []
    dataprime = []
    rows = []
    cols = []
    
    M = ceil(2/h0)
    if M % 2 == 0:
        M += 1
        
    
    h = 2/M
    
#     print(f'h:{h}, h0:{h0}, tau:{tau}, M:{M}, C:{tau/h**2}')
    
    Mop = (M-2)**2
    
    alpha = tau/h**2/2
    
    
    b = np.zeros(Mop)
    
    def add(datasource, val, row, colshift, flag=1):
        """
        Add coefficient to operator.
        """
        datasource.append(val)
        if flag:
            rows.append(row)
            if row+colshift < 0:
                raise Exception(f'Negative col index {row}: {colshift}')
            cols.append(row+colshift)
    
    k = 0
    for row_idx in range(1, M-1):
        for col_idx in range(1, M-1):
            # k = M * (row_idx - 1) + (col_idx - 1)
            # print(f'k: {k} = {row_idx} + {col_idx}')
            
            # Consider Boundary influences
            if is_bc(row_idx + 1, col_idx, M):
#                 print('top')
                b[k] += 0 #boundary_f((row_idx + 1)/M, col_idx/M)
                
            if is_bc(row_idx - 1, col_idx, M):
#                 print('bottom')
                b[k] += 5 #boundary_f((row_idx - 1)/M, col_idx/M)
                
            if is_bc(row_idx, col_idx + 1, M):
#                 print('right')
                b[k] += 0 #boundary_f(row_idx, (col_idx + 1)/M)
                
            if is_bc(row_idx, col_idx - 1, M):
#                 print('left')                
                b[k] += 0 #boundary_f(row_idx, (col_idx - 1)/M)
            
            # Matrix
            if is_bc(row_idx, col_idx, M):
                raise Exception('Adding a bc to matrix - not correct')
            
            # center
            add(data,      1 - 4*alpha, k, 0)
            add(dataprime, 1 + 4*alpha, k, 0, 0)
#             add(4, k, 0)
            
            # left
            if col_idx >= 2:
                add(data,       alpha, k, -1)
                add(dataprime, -alpha, k, -1, 0)
#                 add(-1, k, -1)
            
            # right
            if col_idx < M - 2:
                add(data,       alpha, k, 1)
                add(dataprime, -alpha, k, 1, 0)
                
#                 add(-1, k, 1)
            
            # top
            if row_idx < M - 2:
                add(data,       alpha, k, M - 2)
                add(dataprime, -alpha, k, M - 2, 0)
#                 add(-1, k, M-2)
            
            # bottom
            if row_idx >= 2:
                add(data,       alpha, k, -(M - 2))
                add(dataprime, -alpha, k, -(M - 2), 0)
#                 add(-1, k, -M+2)
            k += 1
            
    # Check for negative column indexes
    if any([x<0 for x in cols]):
        print(cols)
        raise Exception('Negative column index')

    A      = coo_matrix((data,      (rows, cols))).tocsc()
    Aprime = coo_matrix((dataprime, (rows, cols))).tocsc()
    
    # Ensure matrix is square
    if A.shape[0] != A.shape[1]:
        print('shape:', A.shape)
        return A, b
        raise Exception(f'Matrix is not square: {A.shape}')
    
    # Ensure it's the expected size
    if A.shape[0] != Mop:
        raise Exception(f'Matrix wrong size:{A.shape[0]}')
                
    return A, Aprime, b*alpha


def f(t, h, nt):
    tau = t/nt
    
    A, Aprime, bounds = build_CN_system(tau, h)
    u = np.empty_like(bounds)*0
    
#     invAp = linalg.inv(Aprime)

    loc = len(u)//2

    vals = []

    for i in range(nt):
        b = (A @ u + 2*bounds)
        u, info = linalg.cg(Aprime, b)
            # Check convergence status of solution
        if info > 0:
            print(f'Did not converge at i={i}! iter:', info)
        if info < 0:
            print(f'There was an error in cg at i={i}')
        vals.append(u[loc])
        
    
    return u[loc] #, vals#

def secant(g, u0, u1, tol, h, nt, maxiter=1000):
    u_2= u1
    u_1 = u0
    k = 0
    while u_2 - u_1 > tol and k < maxiter:
        f2 = g(u_2, h, nt)
        f1 = g(u_1, h, nt)
        u_new = u_2 - f2*(u_2 - u_1)/(f2 - f1)
        u_1 = u_2
        u_2 = u_new
        print(u_new)
        k += 1
    if k == maxiter:
        print('max iters achieved', k)
    return u_2

def richardson(f, t, h):
    uh = f(t, h)
    u2h = f(t, h/2)
    return  uh + (uh - u2h)/3

# F = lambda T, h, tau: f(T, h, tau)
g = lambda t, h, nt: f(t, h, nt) - 1  #richardson(f, t, h) - 1

secant(g, 0.3, 0.4, 1e-5, 1/200, 100, maxiter=20)

0.4167315883520492
0.42250492525988714
0.42283609911469244
0.4228412639237386


0.4228412639237386